<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_tensorflow2/blob/main/7_PRETRAIN_METHOD/KorNLI_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 마련하기

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/BERT/data_in/

/content/drive/MyDrive/BERT/data_in


In [ ]:
ls

naver_movie/  nsmc/


In [ ]:
!git clone 'https://github.com/kakaobrain/KorNLUDatasets.git'

Cloning into 'KorNLUDatasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.
Checking out files: 100% (9/9), done.


### 허깅페이스 transformer 라이브러리 가져오기

In [ ]:
pip install transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 9.3 MB/s 
     |████████████████████████████████| 880 kB 50.6 MB/s 
     |████████████████████████████████| 3.0 MB 59.8 MB/s 
     |████████████████████████████████| 1.2 MB 57.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e9d6e6f2032481327c812888e9a85605787b4e3b014122d74dce01cda69f104e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


### 데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer

In [ ]:
DATA_IN_PATH = './data_in/KOR'

In [ ]:
TRAIN_XNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'multinli.train.ko.tsv')

multinli_data = pd.read_csv(TRAIN_XNLI_DF, sep='\t', error_bad_lines=False)
multinli_data.head(10)

In [ ]:
print('전체 multinli_data 개수: {}'.format(len(multinli_data)))

In [ ]:
TRAIN_SNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'snli_1.0_train.kor.tsv')

snli_data = pd.read_csv(TRAIN_SNLI_DF, sep='\t', error_bad_lines=False)
snli_data.head(10)

In [ ]:
print('전체 snli_data 개수: {}'.format(len(snli_data)))

In [ ]:
train_data = pd.concat([multinli_data, snli_data], axis=0)
train_data.head(10)

In [ ]:
print('전체 train_data 개수: {}'.format(len(train_data)))

In [ ]:
train_set = pd.Series(train_data['sentence1'].tolist() + train_data['sentence2'].tolist()).astype(str)
train_set.head()